In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import re

In [2]:
class ImageFolderWithPaths(datasets.ImageFolder):
    # override the __getitem__ method. this is the method dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path


In [3]:
TRAIN_DIR = '/Users/alexandruparaschiv/Desktop/cell_images/processed_data/first_train'
TEST_DIR = '/Users/alexandruparaschiv/Desktop/cell_images/processed_data/first_test'

In [4]:
transform = transforms.Compose(
    [   transforms.Resize(size=(64,64)),
        transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [5]:
data_dir = TRAIN_DIR
dataset = ImageFolderWithPaths(data_dir,transform=transform) # our custom dataset
train_data = torch.utils.data.DataLoader(dataset,shuffle=True)
train_data_loader = train_data


data_dir = TEST_DIR
dataset = ImageFolderWithPaths(data_dir,transform=transform) # our custom dataset
test_data = torch.utils.data.DataLoader(dataset,shuffle=True)
test_data_loader = test_data

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 13*13, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)
        self.dropout=nn.Dropout(0.2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x=self.dropout(x)
        x = self.pool(F.relu(self.conv2(x)))
        x=self.dropout(x)
        x = x.view(1, 16*13*13)
        x = F.relu(self.fc1(x))
        x=self.dropout(x)
        x = F.relu(self.fc2(x))
        x=self.dropout(x)
        x = self.fc3(x)
        x=self.dropout(x)
        return x


net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.6)
classes = ['parazitized','uninfected']

In [7]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_data_loader, 0):
        # get the inputs
        inputs, labels, path = data

        match = re.search("uninfected", path[0])
        if not match:
            labels[0] = 0
        elif match:
            labels[0] = 1


        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        #print(i,loss)
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 0.678
[1,  4000] loss: 0.662
[1,  6000] loss: 0.655
[1,  8000] loss: 0.654
[1, 10000] loss: 0.624
[1, 12000] loss: 0.644
[1, 14000] loss: 0.635
[1, 16000] loss: 0.618
[1, 18000] loss: 0.586
[2,  2000] loss: 0.581
[2,  4000] loss: 0.512
[2,  6000] loss: 0.370
[2,  8000] loss: 0.267
[2, 10000] loss: 0.250
[2, 12000] loss: 0.212
[2, 14000] loss: 0.222
[2, 16000] loss: 0.224
[2, 18000] loss: 0.214
Finished Training


In [8]:
counter = 0
true_pred = 0
for i, data in enumerate(test_data_loader, 0):
        # get the inputs
        inputs, labels, path = data


        match = re.search("uninfected", path[0])
        if not match:
            labels[0]= 0
        elif match:
            labels[0]= 1
        outputs = net(inputs)

        _, predicted = torch.max(outputs, 1)
        preds = np.squeeze(predicted.numpy())

        if preds == labels[0].numpy():
            true_pred +=1
        counter += 1
print("Accuracy of {}".format(true_pred/counter))

Accuracy of 0.9233632581563389
